In [1]:

import pandas as pd
import math
import plotly.figure_factory as ff

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import math
import plotly.figure_factory as ff

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional, Union

import requests
from requests.exceptions import RequestException
import time
from bs4 import BeautifulSoup

import numpy as np




import pandas as pd
import re
def clean_seasonal_salaries(data, seasonlist):
    """
    Cleans salary data for specified seasons within the Dead Money section
    
    Parameters:
    data (dict): Input data containing Dead Money section
    seasonlist (list): List of seasons to process
    
    Returns:
    pd.DataFrame: DataFrame with cleaned numerical salary values for all seasons
    """
    if 'Dead Money' not in data:
        return pd.DataFrame()
        
    dead = data['Dead Money'].copy()
    
    for season in seasonlist:
        if season not in dead.columns:
            continue
            
        # Fill NA values
        dead[season] = dead[season].fillna('0')
        
        # Remove 'Ext. Elig.' and strip whitespace
        dead[season] = dead[season].str.replace('Ext. Elig.', '', regex=False).str.strip()
        
        # Convert strings to numeric values
        dead[season] = dead[season].apply(lambda x: convert_salary_string(x))
        
        # Ensure integer type
        dead[season] = dead[season].replace('', 0)
        dead[season] = dead[season].astype(int)
    
    return dead

def convert_salary_string(value):
    """
    Converts a salary string to a pure number, handling percentage suffixes for any size number
    """
    if pd.isna(value) or value == '':
        return '0'
    
    if isinstance(value, (int, float)):
        return str(int(value))
    
    value_str = str(value)
    cutoff=1
    if len(value_str)>15:
        cutoff=2
    
    # Check for special strings
    strings_to_check = ['UFA', 'RFA', 'NA', 'N/A']
    if any(s in value_str for s in strings_to_check):
        return '0'
    
    # First, remove $ and commas
    value_str = value_str.replace('$', '').replace(',', '')
    
    # Find where the decimal point is (indicating start of percentage)
    decimal_index = value_str.find('.')
    if decimal_index != -1:
        # Take everything up to the last 8 digits before the decimal
        # (changed from 7 to 8 to capture the correct number of digits)
        non_decimal_part = value_str[:decimal_index]
      
        value_str = non_decimal_part[:-cutoff]  # Remove last 2 digits instead of 1
 
    # Remove any remaining non-digits
    clean_value = ''.join(c for c in value_str if c.isdigit())
    return clean_value if clean_value else '0'



# Usage

def get_team_data(url: str, timeout: int = 10) -> Tuple[List[pd.DataFrame], List[str]]:
    """
    Fetch and parse HTML tables from the team URL, along with their section headers.
    Returns tuple of (list of dataframes, list of headers)
    """
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all tables and their preceding h2 headers
        tables_data = []
        headers = []
        
        # Get all tables
        tables = soup.find_all('table')
        data_dict={}
        
        for table in tables:
            # Look for the nearest preceding h2
            header = None
            prev_elem = table.find_previous('h2')
            if prev_elem:
                header = prev_elem.get_text(strip=True)
            
            # Parse table into DataFrame
            df = pd.read_html(str(table))[0]
            data_dict[header]=df
        
        return data_dict
        
    except RequestException as e:
        print(f"Error fetching data: {e}")
        return [], []
    except ValueError as e:
        print(f"Error parsing HTML tables: {e}")
        return [], []
# Populate the new DataFrame with player options and team options
def team_books(team):
    print(team)
    nba_team_urls = {
        "ATL": "https://www.spotrac.com/nba/atlanta-hawks/yearly",
        "BOS": "https://www.spotrac.com/nba/boston-celtics/yearly",
        "BKN": "https://www.spotrac.com/nba/brooklyn-nets/yearly",
        "CHA": "https://www.spotrac.com/nba/charlotte-hornets/yearly",
        "CHI": "https://www.spotrac.com/nba/chicago-bulls/yearly",
        "CLE": "https://www.spotrac.com/nba/cleveland-cavaliers/yearly",
        "DAL": "https://www.spotrac.com/nba/dallas-mavericks/yearly",
        "DEN": "https://www.spotrac.com/nba/denver-nuggets/yearly",
        "DET": "https://www.spotrac.com/nba/detroit-pistons/yearly",
        "GSW": "https://www.spotrac.com/nba/golden-state-warriors/yearly",
        "HOU": "https://www.spotrac.com/nba/houston-rockets/yearly",
        "IND": "https://www.spotrac.com/nba/indiana-pacers/yearly",
        "LAC": "https://www.spotrac.com/nba/la-clippers/yearly",
        "LAL": "https://www.spotrac.com/nba/los-angeles-lakers/yearly",
        "MEM": "https://www.spotrac.com/nba/memphis-grizzlies/yearly",
        "MIA": "https://www.spotrac.com/nba/miami-heat/yearly",
        "MIL": "https://www.spotrac.com/nba/milwaukee-bucks/yearly",
        "MIN": "https://www.spotrac.com/nba/minnesota-timberwolves/yearly",
        "NOP": "https://www.spotrac.com/nba/new-orleans-pelicans/yearly",
        "NYK": "https://www.spotrac.com/nba/new-york-knicks/yearly",
        "OKC": "https://www.spotrac.com/nba/oklahoma-city-thunder/yearly",
        "ORL": "https://www.spotrac.com/nba/orlando-magic/yearly",
        "PHI": "https://www.spotrac.com/nba/philadelphia-76ers/yearly",
        "PHX": "https://www.spotrac.com/nba/phoenix-suns/yearly",
        "POR": "https://www.spotrac.com/nba/portland-trail-blazers/yearly",
        "SAC": "https://www.spotrac.com/nba/sacramento-kings/yearly",
        "SAS": "https://www.spotrac.com/nba/san-antonio-spurs/yearly",
        "TOR": "https://www.spotrac.com/nba/toronto-raptors/yearly",
        "UTA": "https://www.spotrac.com/nba/utah-jazz/yearly",
        "WAS": "https://www.spotrac.com/nba/washington-wizards/yearly"
    }
    
    url = nba_team_urls[team.upper()]    
    data=get_team_data(url)


    df = data['Upcoming Deadlines']

  

    salary_df = data['Active Roster']
    



    columns = ['Player']
    for col in salary_df.columns[1:]:
        columns.append(col)
    #print(df.columns)
    df.columns=['Deadline Date', 'Player', 'Type', 'Value']
   
    salary_df.columns =columns

    
    salary_df['Player'] = salary_df['Player'].str.split(' ').str[1:].str.join(' ')

   
    
    #salary_df['Player'] = salary_df['Player'].str.replace('III ', '')
    #salary_df['Player'] = salary_df['Player'].str.replace('II ', '')
    seasons= ['2024-25','2025-26','2026-27','2027-28','2028-29']
    extra_seasons = ['2029-30','2030-31']
    for seas in extra_seasons:
        if seas in salary_df.columns:
            seasons.append(seas)
    # List of strings to check for
    strings_to_check = ['UFA','RFA']
    for season in seasons:
        salary_df[season] = salary_df[season].fillna('')
        




        
        #
    
    # Replace the value with 0 if any of the strings are present


    
    for season in seasons:

        
        salary_df[season] = salary_df[season].str.replace('Ext. Elig.', '', regex=False).str.strip()

    
        salary_df[season] = salary_df[season].apply(lambda x: '0' if any(s in x for s in strings_to_check) else x)
        salary_df[season]=salary_df[season].fillna('0')
        salary_df[season] = salary_df[season].str.split(',').str[0:2].str.join('') +salary_df[season].str.split(',').str[2:3].str.join('').str[0:3]

        #salary_df[season] = salary_df[season].str.replace('Two-Way', '578,577', regex=False).str.strip()


        salary_df[season] = salary_df[season].str.replace(r'\D', '', regex=True)
        
        salary_df[season] = salary_df[season].replace('',0)


    

    
    if 'Dead Money' in data.keys():
        seasonlist=['2024-25','2025-26','2026-27','2027-28','2028-29','2029-30']

        dead = clean_seasonal_salaries(data, seasonlist)


        alldead=pd.DataFrame()
        
       
      
        for season in seasonlist:
            if season in dead.columns:
                alldead[season]=[dead[season].sum()]
        alldead['Player'] = ['Dead Cap']

        alldead.reset_index(inplace=True)

        salary_df=pd.concat([salary_df,alldead])
        

    #salary_df = pd.concat([salary_df,cap_hold]).reset_index(drop=True)
    players = salary_df['Player'].unique()
    data=[]

    for player in players:
    
        player_data = df[df['Player'] == player]
        row = {'Player': player}
        for season in seasons:
            if season in salary_df.columns:
                row[season] = 0
                season_data = player_data[player_data['Type'].str.contains(season)]
                if not season_data.empty:
        
                    if 'PLAYER' in season_data['Type'].values[0]:
                        row[season] = 'P'
                    elif 'CLUB' in  season_data['Type'].values[0]:
                        row[season] = 'T'
                    elif 'GUARANTEED' in season_data['Type'].values[0]:
                        row[season]='NG'
                    elif 'EXTENSION' in season_data['Type'].values[0]:
                        row[season]='EE'
                    elif 'RFA' in season_data['Type'].values[0]:
                        row[season]='RFA'
                    elif 'UNREST' in season_data['Type'].values[0]:
                        row[season]='UFA'
                    else:
                    
                        row[season] = season_data['Type'].values[0] + (' ' + season_data['Value'].values[0] if not pd.isna(season_data['Value'].values[0]) else '')
                data.append(row)
            
    
    
    # Display the new organized DataFrame
    new_df = pd.DataFrame(columns=['Player'] + seasons,data=data)
    new_df=new_df.drop_duplicates().reset_index(drop=True)
    salary_df=salary_df.drop_duplicates().reset_index(drop=True)

    #new_df = pd.concat([new_df,cap_options]).reset_index(drop=True)
    for season in seasons:
        salary_df[season] = salary_df[season].astype(float)

    #salary_df['Team']='GSW'
    #new_df['Team']='GSW'
    

    
    salary_df['Team']=team
    new_df['Team']=team

    return salary_df,new_df
teams = ['ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
#teams=['OKC','BKN']
salary=[]
options=[]
for team in teams:
    salary_df,option_df =team_books(team)
    salary.append(salary_df)
    options.append(option_df)

salary_df = pd.concat(salary)
option_df = pd.concat(options)
#salary_df.to_csv('salary.csv',index=False)
option_df

ATL
BOS
BKN
CHA
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHX
POR
SAC
SAS
TOR
UTA
WAS


,Player,2024-25,2025-26,2026-27,2027-28,2028-29,2029-30,2030-31,Team
0,Trae Young,0,0,P,0,0,0,0.0,ATL
1,Clint Capela,0,0,0,0,0,0,0.0,ATL
2,Caris LeVert,0,0,0,0,0,0,0.0,ATL
3,Onyeka Okongwu,0,EE,0,0,0,0,0.0,ATL
4,Zaccharie Risacher,0,0,T,T,RFA,0,0.0,ATL
...,...,...,...,...,...,...,...,...,...
14,Jaylen Nowell,0,0,0,0,0,0,NaN,WAS
15,Justin Champagnie,RFA,0,0,0,0,0,NaN,WAS
16,Jaylen Martin,0,RFA,0,0,0,0,NaN,WAS
17,Tristan Vukcevic,0,RFA,0,0,0,0,NaN,WAS


In [2]:
test1 = "$2,087,5191.5%"
test2 = "$39,256,08327.9%"
print(convert_salary_string(test1))  # Should be '2087519'
print(convert_salary_string(test2))

2087519
39256083


In [3]:
temp_df=pd.DataFrame()
temp_df['Player'] = option_df['Player']
seasons = ['2024-25','2025-26','2026-27','2027-28','2028-29']
for season in seasons:
    temp_df[season] = np.where(option_df[season]!='T', 1, 0)


guar = pd.DataFrame()
guar['Player'] = salary_df['Player']


guar['Guaranteed'] = 0

print(guar)
print(temp_df)
print(salary_df)
for season in seasons:
    guar['Guaranteed']+= temp_df[season]* salary_df[season]
salary_df = salary_df.merge(guar,on='Player')
salary_df.sort_values(by='Guaranteed',inplace=True)
salary_df
salary_df=salary_df.drop_duplicates(subset=['Player','Team'])
salary_df
option_df=option_df.drop_duplicates(subset=['Player','Team'])
option_df


#salary_df.loc[salary_df['Player'].str.contains('Branden Carlson'), '2024-25'] = 990895

option_df.loc[option_df['Player'].str.contains('Scottie Barnes'), '2025-26'] = 0
option_df.loc[option_df['Player'].str.contains('Bradley Beal'), '2026-27'] = 'P'
option_df.loc[option_df['Player'].str.contains('Jalen Brunson'), '2024-25'] = 0
option_df.loc[option_df['Player'].str.contains('Jalen Brunson'), '2025-26'] = 0
option_df.loc[option_df['Player'].str.contains('Julius Randle'), '2026-27'] = 'P'

                Player  Guaranteed
0           Trae Young           0
1         Clint Capela           0
2         Caris LeVert           0
3       Onyeka Okongwu           0
4   Zaccharie Risacher           0
..                 ...         ...
14       Jaylen Nowell           0
15   Justin Champagnie           0
16       Jaylen Martin           0
17    Tristan Vukcevic           0
18            Dead Cap           0

[543 rows x 2 columns]
                Player  2024-25  2025-26  2026-27  2027-28  2028-29
0           Trae Young        1        1        1        1        1
1         Clint Capela        1        1        1        1        1
2         Caris LeVert        1        1        1        1        1
3       Onyeka Okongwu        1        1        1        1        1
4   Zaccharie Risacher        1        1        0        0        1
..                 ...      ...      ...      ...      ...      ...
14       Jaylen Nowell        1        1        1        1        1
15   Justin 

In [4]:
salary_df.to_csv('salary.csv',index=False)
option_df.to_csv('option.csv',index=False)
#salary_df.to_csv('../data/salary.csv',index=False)
#option_df.to_csv('../data/option.csv',index=False)
salary_df[salary_df.Team=='OKC']

,Player,Pos,Age,2024-25,2025-26,2026-27,2027-28,2028-29,2029-30,2030-31,index,Team,Guaranteed
890,Branden Carlson,C,25.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OKC,0.0
891,Alex Ducas,G,24.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OKC,0.0
892,Adam Flagler,G,25.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OKC,0.0
375,Dead Cap,NaN,NaN,3792452.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,OKC,553591.0
887,Ajay Mitchell,PG,22.0,3000000.0,3000000.0,0.0,0.0,0.0,0.0,NaN,NaN,OKC,3000000.0
889,Jaylin Williams,C,22.0,2019699.0,2187699.0,0.0,0.0,0.0,0.0,NaN,NaN,OKC,4207398.0
888,Dillon Jones,SF,23.0,2622360.0,2753280.0,2884440.0,5200646.0,0.0,0.0,NaN,NaN,OKC,5375640.0
885,Nikola Topic,PG,19.0,4935960.0,5182920.0,5429760.0,7482210.0,0.0,0.0,NaN,NaN,OKC,10118880.0
886,Jalen Williams,SF,23.0,4775760.0,6580997.0,0.0,0.0,0.0,0.0,NaN,NaN,OKC,11356757.0
883,Cason Wallace,PG,21.0,5555880.0,5820240.0,7420806.0,0.0,0.0,0.0,NaN,NaN,OKC,11376120.0


In [5]:
'''
dfs =pd.read_html('https://basketball.realgm.com/nba/info/salary_cap')
cap = dfs[0]
cap.columns = cap.columns.droplevel()
cap.columns
columns = [ 'Salary Cap', 'Luxury Tax', '1st Apron', '2nd Apron', 'BAE',
       'Standard / Non-Taxpayer', 'Taxpayer', 'Team Room / Under Cap']
for col in columns:
    cap[col] = cap[col].str.replace(r'\D', '', regex=True)
    cap[col] = cap[col].astype(float)
#cap.to_csv('../data/cap.csv',index=False)
'''

"\ndfs =pd.read_html('https://basketball.realgm.com/nba/info/salary_cap')\ncap = dfs[0]\ncap.columns = cap.columns.droplevel()\ncap.columns\ncolumns = [ 'Salary Cap', 'Luxury Tax', '1st Apron', '2nd Apron', 'BAE',\n       'Standard / Non-Taxpayer', 'Taxpayer', 'Team Room / Under Cap']\nfor col in columns:\n    cap[col] = cap[col].str.replace(r'\\D', '', regex=True)\n    cap[col] = cap[col].astype(float)\n#cap.to_csv('../data/cap.csv',index=False)\n"

In [6]:
'''
df = pd.read_csv('../data/lebron.csv')
jrue =df[df['NBA ID']==201950]

df =df[df['NBA ID']!=201950]


jrue['Player'] = 'jrue holiday'

df =pd.concat([df,jrue]).reset_index(drop=True)
df.sort_values(['year','Player'],inplace=True)
df.to_csv('../data/lebron.csv',index=False)
'''

"\ndf = pd.read_csv('../data/lebron.csv')\njrue =df[df['NBA ID']==201950]\n\ndf =df[df['NBA ID']!=201950]\n\n\njrue['Player'] = 'jrue holiday'\n\ndf =pd.concat([df,jrue]).reset_index(drop=True)\ndf.sort_values(['year','Player'],inplace=True)\ndf.to_csv('../data/lebron.csv',index=False)\n"

In [7]:
'''
cap=pd.read_csv('../data/cap.csv')
cap['year'] = cap['Season'].str.split('-').str[1:].str.join('')
cap['year'] = cap['year'].astype(int)
cap.to_csv('../data/cap.csv',index=False)
'''

"\ncap=pd.read_csv('../data/cap.csv')\ncap['year'] = cap['Season'].str.split('-').str[1:].str.join('')\ncap['year'] = cap['year'].astype(int)\ncap.to_csv('../data/cap.csv',index=False)\n"

In [8]:

# Let's test both cases
test1 = "$2,087,5191.5%"
test2 = "$39,256,08327.9%"
print(convert_salary_string(test1))  # Should be '2087519'
print(convert_salary_string(test2))  # Should be '39256083'

2087519
39256083
